In [ ]:
#i "nuget:https://pkgs.dev.azure.com/dnceng/public/_packaging/dotnet5/nuget/v3/index.json" 
#i "nuget:https://pkgs.dev.azure.com/dnceng/public/_packaging/dotnet-tools/nuget/v3/index.json" 
#r "nuget: BserClient, 1.3.0"
#r "nuget: Plotly.NET, 2.0.0-preview.6"
#r "nuget: Plotly.NET.Interactive, 2.0.0-preview.6"

Restore sources https://pkgs.dev.azure.com/dnceng/public/_packaging/dotnet-tools/nuget/v3/index.json https://pkgs.dev.azure.com/dnceng/public/_packaging/dotnet5/nuget/v3/index.json

In [ ]:
using System.Text.Json;
using System.Text.Json.Serialization;
using System.IO;
using BserClient;
using System.Collections.Generic;

string readText = File.ReadAllText("games.json");

List<BserClient.Types.UserGameObj> games = 
JsonSerializer.Deserialize<List<BserClient.Types.UserGameObj>>(readText);

// List<dynamic> games = JsonSerializer.Deserialize<List<dynamic>>(readText);
// var games = Json.Decode(readText);
int ohioGames = 0;
int koreaGames = 0;
foreach (var game in games)
{
    if (game.matchingMode == 3 && game.seasonId == 3) {
        if (game.serverName == "Ohio") ohioGames += 1;
        if (game.serverName == "Seoul")  koreaGames += 1;
    }
}

// get ohioGames vs korea games
Console.WriteLine(ohioGames);
Console.WriteLine(koreaGames);

498


327


In [ ]:

using System.IO;
using System.Linq;
using Microsoft.FSharp.Core;
using Plotly.NET;
using static Plotly.NET.GenericChartExtensions;
using Microsoft.FSharp.Collections;
using Microsoft.AspNetCore.Html;
void addOrUpdate(Dictionary<string, int> dic, string key, int newValue)
{
    int val;
    if (dic.TryGetValue(key, out val))
    {
        // yay, value exists!
        dic[key] = val + newValue;
    }
    else
    {
        // darn, lets add the value
        dic.Add(key, newValue);
    }
}

public class GameKillerCharactersData
{
    public string name { get; set; }
    public int characterNum { get; set; }
    public float mmrAvg {get; set;}
    public string killDetail {get; set;}
    public string causeOfDeath {get; set;}
    public string placeOfDeath {get; set;}
    public string killerCharacter {get; set;} 
    public string killerWeapon {get; set;} 
    public string killerUserNum2 {get; set;}
    public string killerUserNum3 {get; set;}
}

public class Csv
{
    /// <summary>
    ///  Generates csv file from a list of items from generic list
    /// </summary>
    public static string GenerateReport<T>(List<T> items) where T : class
    {
        var output = "";
        var delimiter = ",";
        var properties = typeof(T).GetProperties()
            .Where(n =>
                n.PropertyType == typeof(string)
                || n.PropertyType == typeof(bool)
                || n.PropertyType == typeof(char)
                || n.PropertyType == typeof(byte)
                || n.PropertyType == typeof(decimal)
                || n.PropertyType == typeof(int)
            );
        using (var sw = new StringWriter())
        {
            var header = properties
                .Select(n => n.Name)
                .Aggregate((a, b) => a + delimiter + b);
            sw.WriteLine(header);
            foreach (var item in items)
            {
                var row = properties
                    .Select(n => n.GetValue(item, null))
                    .Select(n => n == null ? null : n.ToString())
                    .Aggregate((a, b) => a + delimiter + b);
                sw.WriteLine(row);
            }
            output = sw.ToString();
        }
        return output;
    }
}


List<GameKillerCharactersData> killData = new List<GameKillerCharactersData> {};
// go through each character

string json = System.IO.File.ReadAllText(@"data/Character.json");
System.Text.Json.JsonElement characterDict = System.Text.Json.JsonSerializer.Deserialize<System.Text.Json.JsonElement>(json);
System.Text.Json.JsonElement characterData = characterDict.GetProperty("data");
// TODO count who kills me in general with another dictionary 
foreach (var gameGroup in games.GroupBy(x => x.characterNum))
{
    // counts how many deaths I have 
    Dictionary<string, int> killedByTimes =
    new Dictionary<string, int>();
    var characterIndex = Convert.ToInt32(gameGroup.Key-1);
    System.Text.Json.JsonElement selectedCharacterData = characterData[characterIndex];
    // Console.WriteLine(selectedCharacterData);
    var characterName = selectedCharacterData.GetProperty("name");
    // Console.WriteLine("Info for characterNum: " + characterName);
    foreach (BserClient.Types.UserGameObj game in gameGroup) {
        // ignore games where I place first
        if (game.gameRank != 1) {
            GameKillerCharactersData gameData = new GameKillerCharactersData();
            String killerCharacter = game.killerCharacter;
            if(killerCharacter != "" && killerCharacter != null) {
                addOrUpdate(killedByTimes, killerCharacter, 1);
            }
        }
    }
    // killedByTimes.Select(i => $"{i.Key}: {i.Value}").ToList().ForEach(Console.WriteLine);

    // Old looping mechanism
    // foreach (KeyValuePair<string, int> kvp in killedByTimes)
    // {
    //     // print next table instead
    //     // https://stackoverflow.com/questions/856845/how-to-best-way-to-draw-table-in-console-app-c
    //     Console.WriteLine("Killed by: " + kvp);
    //     if (kvp.Key != null) {
    //         //textBox3.Text += ("Key = {0}, Value = {1}", kvp.Key, kvp.Value);
    //         // Console.WriteLine("Key = {0}, Value = {1}", kvp.Key, kvp.Value);
    //     }
    // }
  
    // var values = new List<int> {1,2,3} as IEnumerable<int>; 
    // var keys = new List<string> {"Product A", "Product B", "Product C"} as IEnumerable<string>;
    // var bar = Chart.Bar(values,keys);
    // figure out how to map data to 
    // string[] x = new string[] { "test", "test2" };
    var x = killedByTimes.Keys.ToArray();
    var y = killedByTimes.Values.ToArray();

    Axis.LinearAxis xAxis = new Axis.LinearAxis();
    xAxis.SetValue("title", "xAxis");
    xAxis.SetValue("showgrid", false);
    xAxis.SetValue("showline", true);

    Axis.LinearAxis yAxis = new Axis.LinearAxis();
    yAxis.SetValue("title", "yAxis");
    yAxis.SetValue("showgrid", false);
    yAxis.SetValue("showline", true);

    Layout layout = new Layout();
    layout.SetValue("xaxis", xAxis);
    layout.SetValue("yaxis", yAxis);
    layout.SetValue("title", "Died to as " + characterName);
    
    Trace trace = new Trace("bar");
    trace.SetValue("x", x);
    trace.SetValue("y", y);
    trace.SetValue("mode", "markers");
    // trace.SetValue("name", characterName);

    var chart = GenericChart
        .ofTraceObject(trace)
        .WithLayout(layout);

    // figure out how to save static files
    var data = GenericChart.toChartHTML(chart);
    DisplayExtensions.DisplayAs(data,"text/html");
 
    // log results

    // eventsInYear.Key - year
    // eventsInYear - collection of events in that year
}

// var csvEntries = Csv.GenerateReport<GameKillerCharactersData>(killData);

//  // diff list if exists
//  System.IO.File.WriteAllText("kills.csv", csvEntries);


<!-- Plotly chart will be drawn inside this DIV -->

<!-- Plotly chart will be drawn inside this DIV -->

<!-- Plotly chart will be drawn inside this DIV -->

<!-- Plotly chart will be drawn inside this DIV -->

<!-- Plotly chart will be drawn inside this DIV -->

<!-- Plotly chart will be drawn inside this DIV -->

<!-- Plotly chart will be drawn inside this DIV -->

<!-- Plotly chart will be drawn inside this DIV -->

<!-- Plotly chart will be drawn inside this DIV -->